In [23]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.2f}'.format

In [24]:
# reading dataset for Indonesia IPUMS
data_indonesia = pd.read_stata('/users/quasar/downloads/ipumsi_00030.dta')
data_indonesia.groupby('year').count()

,country,sample,serial,hhwt,pernum,perwt,age,sex,edattain,edattaind,yrschool,educid,empstat,empstatd,indgen,classwk,classwkd,incwage
year,,,,,,,,,,,,,,,,,,
1971,526746,526746,526746,526746,526746,526746,526746,526746,526746,526746,0,526746,526746,526746,526746,526746,526746,0
1976,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364,234364
1980,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,6091135,0
1985,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,515826,0
1990,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,789701,0
1995,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476,629476
2000,17672340,17672340,17672340,17672340,17672340,17672340,17672340,17672340,17672340,17672340,0,17672340,0,0,0,17672340,17672340,0
2005,964964,964964,964964,964964,964964,964964,964964,964964,964964,964964,964964,964964,0,0,964964,964964,964964,0
2010,20747049,20747049,20747049,20747049,20747049,20747049,20747049,20747049,20747049,20747049,0,20747049,20747049,20747049,20747049,20747049,20747049,0


## Sector in economy - Agriculture, Industry, Unskilled services, Skilled services 

In [25]:
data_indonesia.head()

,country,year,sample,serial,hhwt,pernum,perwt,age,sex,edattain,edattaind,yrschool,educid,empstat,empstatd,indgen,classwk,classwkd,incwage
0,indonesia,1971,indonesia 1971,"1,000.00",661,1,608,30,male,primary completed,primary (6 yrs) completed,NaN,"primary education (6 years), completed",employed,"employed, not specified",construction,wage/salary worker,wage/salary worker,NaN
1,indonesia,1971,indonesia 1971,"1,000.00",661,2,608,25,female,less than primary completed,some primary completed,NaN,"primary education (6 years), not completed",inactive,housework,niu (not in universe),niu (not in universe),niu (not in universe),NaN
2,indonesia,1971,indonesia 1971,"2,000.00",660,1,608,35,male,less than primary completed,no schooling,NaN,none,employed,"employed, not specified","agriculture, fishing, and forestry",wage/salary worker,wage/salary worker,NaN
3,indonesia,1971,indonesia 1971,"2,000.00",660,2,608,30,female,less than primary completed,no schooling,NaN,none,inactive,housework,niu (not in universe),niu (not in universe),niu (not in universe),NaN
4,indonesia,1971,indonesia 1971,"2,000.00",660,3,608,12,male,less than primary completed,no schooling,NaN,none,employed,"employed, not specified",public administration and defense,unpaid worker,unpaid family worker,NaN


In [26]:
data_indonesia.isnull().sum()

country             0
year                0
sample              0
serial              0
hhwt                0
pernum              0
perwt               0
age                 0
sex                 0
edattain            0
edattaind           0
yrschool     38946135
educid              0
empstat      18637304
empstatd     18637304
indgen       17672340
classwk             0
classwkd            0
incwage      47307761
dtype: int64

In [7]:
"""
data_indonesia_clean = data_indonesia.loc[(data_indonesia['year'] == '1971') | (data_indonesia['year'] == '1995')]
data_indonesia_clean.groupby('year').count()
"""

"\ndata_indonesia_clean = data_indonesia.loc[(data_indonesia['year'] == '1971') | (data_indonesia['year'] == '1995')]\ndata_indonesia_clean.groupby('year').count()\n"

In [27]:
data_indonesia['indgen'].replace(to_replace=['agriculture, fishing, and forestry'], value='Agriculture', inplace=True)
data_indonesia['indgen'].replace(to_replace=['construction', 'manufacturing' ,'mining and extraction'], value='Industry', inplace=True)
data_indonesia['indgen'].replace(to_replace=['hotels and restaurants','private household services',
                    'transportation, storage, and communications', 'wholesale and retail trade'], 
                    value='Unskilled service', inplace=True)
data_indonesia['indgen'].replace(to_replace=['education', 'financial services and insurance','health and social work',
                    'public administration and defense',
                    'other services', 'business services and real estate', 
                    'electricity, gas, water and waste management' ], 
                    value='Skilled service', inplace=True)

indexNames = data_indonesia[data_indonesia['indgen'].isin(['unknown', 'niu (not in universe)'])].index
data_indonesia = data_indonesia.drop(indexNames)
data_indonesia['indgen'].replace(to_replace=['niu (not in universe)', 'unknown'], value='Agriculture', inplace=True)
print("\nSector by labour share % ")
share = data_indonesia.groupby('year')['indgen'].value_counts(normalize=True)
share


Sector by labour share % 


year                   
1971  Agriculture         0.37
      Unskilled service   0.33
      Skilled service     0.17
      Industry            0.13
1976  Agriculture         0.53
      Unskilled service   0.27
      Industry            0.11
      Skilled service     0.10
1980  Agriculture         0.56
      Unskilled service   0.23
      Industry            0.13
      Skilled service     0.08
1985  Agriculture         0.56
      Unskilled service   0.22
      Industry            0.13
      Skilled service     0.09
1990  Agriculture         0.51
      Unskilled service   0.22
      Industry            0.16
      Skilled service     0.11
1995  Agriculture         0.46
      Unskilled service   0.23
      Industry            0.19
      Skilled service     0.12
2000  Agriculture          NaN
      Industry             NaN
      Unskilled service    NaN
      Skilled service      NaN
2005  Agriculture         0.50
      Unskilled service   0.25
      Industry            0.12
      Skilled s

## Mean of incwage (Wage) by year and sector

In [28]:
data_indonesia.educid.unique()

['primary education (6 years), completed', 'none', 'primary education (6 years), not completed', 'junior high school', 'junior vocational school', ..., 'diploma i/ii', 'graduate school, completed', 'graduate school, not completed', 'graduate school', 'niu (not in universe)']
Length: 69
Categories (69, object): ['niu (not in universe)' < 'none' < 'no schooling or primary not completed' < 'preschool, not completed' ... 'graduate school, completed' < 'masters' < 'doctoral program' < 'unknown']

In [29]:
print("\nMean of incwage by year and sector\n")
print(data_indonesia.groupby(['year','indgen', 'educid'])['incwage'].mean())


Mean of incwage by year and sector

year  indgen           educid                               
1971  Agriculture      niu (not in universe)                   NaN
                       none                                    NaN
                       no schooling or primary not completed   NaN
                       preschool, not completed                NaN
                       preschool, completed                    NaN
                                                                ..
2010  Skilled service  graduate school, not completed          NaN
                       graduate school, completed              NaN
                       masters                                 NaN
                       doctoral program                        NaN
                       unknown                                 NaN
Name: incwage, Length: 2484, dtype: float64


## Median of incwage by year and sector 

In [22]:
print("\nMean of incwage by year and sector\n")
print(data_indonesia.groupby(['year','indgen', 'classwk'])['incwage'].mean())


Mean of incwage by year and sector

year  indgen             classwk              
1971  Agriculture        niu (not in universe)            NaN
                         self-employed                    NaN
                         wage/salary worker               NaN
                         unpaid worker                    NaN
                         unknown/missing                  NaN
      Industry           niu (not in universe)            NaN
                         self-employed                    NaN
                         wage/salary worker               NaN
                         unpaid worker                    NaN
                         unknown/missing                  NaN
      Unskilled service  niu (not in universe)            NaN
                         self-employed                    NaN
                         wage/salary worker               NaN
                         unpaid worker                    NaN
                         unknown/missing        

In [30]:
print("\nMedian of incwage by year and sector\n")
print(data_indonesia.groupby(['year','indgen', 'educid'])['incwage'].median())


Median of incwage by year and sector

year  indgen           educid                               
1971  Agriculture      niu (not in universe)                   NaN
                       none                                    NaN
                       no schooling or primary not completed   NaN
                       preschool, not completed                NaN
                       preschool, completed                    NaN
                                                                ..
2010  Skilled service  graduate school, not completed          NaN
                       graduate school, completed              NaN
                       masters                                 NaN
                       doctoral program                        NaN
                       unknown                                 NaN
Name: incwage, Length: 2484, dtype: float64


In [31]:
data_indonesia['educid'].unique()

['primary education (6 years), completed', 'none', 'primary education (6 years), not completed', 'junior high school', 'junior vocational school', ..., 'diploma i/ii', 'graduate school, completed', 'graduate school, not completed', 'graduate school', 'niu (not in universe)']
Length: 69
Categories (69, object): ['niu (not in universe)' < 'none' < 'no schooling or primary not completed' < 'preschool, not completed' ... 'graduate school, completed' < 'masters' < 'doctoral program' < 'unknown']

In [32]:
data_indonesia['yrschool'].unique()

[NaN, '6 years', '12 years', '15 years', 'some primary', ..., '8 years', '14 years', 'some tertiary', '13 years', '17 years']
Length: 23
Categories (22, object): ['none or pre-school' < '1 year' < '2 years' < '3 years' ... 'some primary' < 'some secondary' < 'some tertiary' < 'unknown/missing']

In [33]:
data_indonesia['yrschool'].replace(to_replace= ['none or pre-school'], value = 0, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['1 year'], value = 1, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['2 years'], value = 2, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['3 years'], value = 3 , inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['4 years'], value = 4, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['5 years'], value = 5, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['6 years'], value = 6 , inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['7 years'], value = 7, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['8 years'], value = 8, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['9 years'], value = 9 , inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['10 years'], value = 10, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['11 years'], value = 11, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['12 years'], value = 12, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['13 years'], value = 13, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['14 years'], value = 14, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['15 years'], value = 15, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['16 years'], value = 16, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['17 years'], value = 17, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['18 years or more'], value = 18 , inplace=True)

data_indonesia['yrschool'].replace(to_replace= ['not specified'], value = 90, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['some primary'], value = 91, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['some technical after primary'], value = 92, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['some secondary'], value = 93, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['some tertiary'], value = 94, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['adult literacy'], value = 95, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['special education'], value = 96, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['unknown/missing'], value = 98, inplace=True)
data_indonesia['yrschool'].replace(to_replace= ['niu (not in universe)'], value = 99, inplace=True)

In [34]:
data_indonesia['yrschool'].unique()

[NaN, 6, 12, 15, 91, ..., 8, 14, 94, 13, 17]
Length: 23
Categories (22, int64): [0 < 1 < 2 < 3 ... 91 < 93 < 94 < 98]

In [35]:
data_indonesia['indgen'].unique()

['Industry', 'Agriculture', 'Skilled service', 'Unskilled service', NaN]
Categories (4, object): ['Agriculture' < 'Industry' < 'Unskilled service' < 'Skilled service']

In [36]:
data_indonesia.shape

(32102950, 19)

In [37]:
# calculting mincer returns 
# use sector dummy and sex as control 

data_reg = pd.get_dummies(data_indonesia, columns=['indgen', 'sex'], drop_first=True)
data_reg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32102950 entries, 0 to 48171596
Data columns (total 21 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   country                   category
 1   year                      category
 2   sample                    category
 3   serial                    float64 
 4   hhwt                      int16   
 5   pernum                    int8    
 6   perwt                     int16   
 7   age                       category
 8   edattain                  category
 9   edattaind                 category
 10  yrschool                  category
 11  educid                    category
 12  empstat                   category
 13  empstatd                  category
 14  classwk                   category
 15  classwkd                  category
 16  incwage                   float64 
 17  indgen_Industry           uint8   
 18  indgen_Unskilled service  uint8   
 19  indgen_Skilled service    uint8   
 20  

In [38]:
# transforming variables in educin 
"""
data_reg['educin'].replace(to_replace=['illiterate, less than primary'], value = 0 , inplace=True)
data_reg['educin'].replace(to_replace=['literate, less than primary'], value = 2 , inplace=True)
data_reg['educin'].replace(to_replace=['primary'], value = 5 , inplace=True)
data_reg['educin'].replace(to_replace=['middle'], value = 8 , inplace=True)
data_reg['educin'].replace(to_replace=['secondary'], value = 10 , inplace=True)
data_reg['educin'].replace(to_replace=['higher secondary'], value = 12 , inplace=True)
data_reg['educin'].replace(to_replace=['undergraduate or graduate'], value = 15 , inplace=True)
data_reg['educin'].replace(to_replace=['unknown'], value = 99 , inplace=True)
"""

"\ndata_reg['educin'].replace(to_replace=['illiterate, less than primary'], value = 0 , inplace=True)\ndata_reg['educin'].replace(to_replace=['literate, less than primary'], value = 2 , inplace=True)\ndata_reg['educin'].replace(to_replace=['primary'], value = 5 , inplace=True)\ndata_reg['educin'].replace(to_replace=['middle'], value = 8 , inplace=True)\ndata_reg['educin'].replace(to_replace=['secondary'], value = 10 , inplace=True)\ndata_reg['educin'].replace(to_replace=['higher secondary'], value = 12 , inplace=True)\ndata_reg['educin'].replace(to_replace=['undergraduate or graduate'], value = 15 , inplace=True)\ndata_reg['educin'].replace(to_replace=['unknown'], value = 99 , inplace=True)\n"

In [39]:
data_reg['year'].unique()

['1971', '1976', '1980', '1985', '1990', '1995', '2000', '2005', '2010']
Categories (9, object): ['1971' < '1976' < '1980' < '1985' ... '1995' < '2000' < '2005' < '2010']

In [40]:
data_reg.yrschool.unique()

[NaN, 6, 12, 15, 91, ..., 8, 14, 94, 13, 17]
Length: 23
Categories (22, int64): [0 < 1 < 2 < 3 ... 91 < 93 < 94 < 98]

In [41]:
data_reg = data_reg.loc[data_reg.yrschool.isin([4,  0,  2,  3,  1,  6,  5, 12, 11,  8,  9, 15,  7, 13, 10,
       14, 17, 18, 16])]

In [44]:
# Income 
data_reg =  data_reg[data_reg['incwage'] != 0]

Y = data_reg[['year', 'incwage']]
X = data_reg[['year', 'sex_female', 'yrschool','indgen_Industry', 
                    'indgen_Unskilled service', 'indgen_Skilled service']]

# year 1976
Y_1976 = Y.loc[Y.year == '1976']['incwage']
X_1976 = data_reg.loc[data_reg.year == '1976']
X_1976_1 = X_1976[['sex_female',
                     'indgen_Industry', 'indgen_Unskilled service', 'indgen_Skilled service']]

print(X_1976_1.shape)
print(Y_1976.shape)

# regression of log wage on sector and sex 
reg_1 = sm.OLS(endog= np.log(Y_1976), exog= sm.add_constant(X_1976_1))
result_1 = reg_1.fit()
print(result_1.summary())

# for calculting mincer returns - adding years of schooling 
X_1976_2 = X_1976[['sex_female', 'yrschool',
                     'indgen_Industry', 'indgen_Unskilled service', 'indgen_Skilled service']]

reg_2 = sm.OLS(endog= np.log(Y_1976), exog= sm.add_constant(X_1976_2))
result_2 = reg_2.fit()
print(result_2.summary())

(102130, 4)
(102130,)
                            OLS Regression Results                            
Dep. Variable:                incwage   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     7528.
Date:                Sun, 19 Dec 2021   Prob (F-statistic):               0.00
Time:                        19:19:15   Log-Likelihood:            -2.5087e+05
No. Observations:              102130   AIC:                         5.018e+05
Df Residuals:                  102125   BIC:                         5.018e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
co

/usr/local/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                incwage   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     6032.
Date:                Sun, 19 Dec 2021   Prob (F-statistic):               0.00
Time:                        19:19:15   Log-Likelihood:            -2.5085e+05
No. Observations:              102130   AIC:                         5.017e+05
Df Residuals:                  102124   BIC:                         5.018e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

/usr/local/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [45]:
# year 1995
Y_1995 = Y.loc[Y.year == '1995']['incwage']
X_1995 = data_reg.loc[data_reg.year == '1995']
X_1995_1 = X_1995[['sex_female',
                     'indgen_Industry', 'indgen_Unskilled service', 'indgen_Skilled service']]

print(X_1995_1.shape)
print(Y_1995.shape)

# regression of log wage on sector and sex 
reg_3 = sm.OLS(endog= np.log(Y_1995), exog= sm.add_constant(X_1995_1))
result_3 = reg_3.fit()
print(result_3.summary())

# for calculting mincer returns - adding years of schooling 
X_1995_2 = X_1995[['sex_female', 'yrschool',
                     'indgen_Industry', 'indgen_Unskilled service', 'indgen_Skilled service']]

reg_4 = sm.OLS(endog= np.log(Y_1995), exog= sm.add_constant(X_1995_2))
result_4 = reg_4.fit()
print(result_4.summary())

(273753, 4)
(273753,)
                            OLS Regression Results                            
Dep. Variable:                incwage   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                 2.957e+04
Date:                Sun, 19 Dec 2021   Prob (F-statistic):               0.00
Time:                        19:19:43   Log-Likelihood:            -5.3363e+05
No. Observations:              273753   AIC:                         1.067e+06
Df Residuals:                  273748   BIC:                         1.067e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
co

/usr/local/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                incwage   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                 2.447e+04
Date:                Sun, 19 Dec 2021   Prob (F-statistic):               0.00
Time:                        19:19:43   Log-Likelihood:            -5.3221e+05
No. Observations:              273753   AIC:                         1.064e+06
Df Residuals:                  273747   BIC:                         1.064e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [50]:
from statsmodels.iolib.summary2 import summary_col

results_table  = summary_col(results = [result_1, result_2, result_3, result_4],
                              float_format= '%0.2f',
                              model_names=['1976(unadj.)', '1976(adj.)',
                                           '1995(unadj.)', '1995(adj.)',
                                           ],
                              regressor_order=["const" ,"sex_female", "yrschool", 
                                                  "indgen_Industry", "indgen_Unskilled service",
                                                  "indgen_Skilled service"],
                              drop_omitted=True,
                              stars = True)
results_table.add_title(title = "Indonesia : Mincer Return - Results from Earnings Regressions", results= None)
print(results_table)

     Indonesia : Mincer Return - Results from Earnings Regressions
                         1976(unadj.) 1976(adj.) 1995(unadj.) 1995(adj.)
------------------------------------------------------------------------
const                    15.08***     15.14***   15.41***     15.66***  
                         (0.01)       (0.02)     (0.01)       (0.01)    
sex_female               0.25***      0.23***    0.28***      0.23***   
                         (0.02)       (0.02)     (0.01)       (0.01)    
yrschool                              -0.02***                -0.05***  
                                      (0.00)                  (0.00)    
indgen_Industry          -2.95***     -2.93***   -2.10***     -1.97***  
                         (0.03)       (0.03)     (0.01)       (0.01)    
indgen_Unskilled service -1.27***     -1.24***   -0.37***     -0.24***  
                         (0.02)       (0.02)     (0.01)       (0.01)    
indgen_Skilled service   -4.74***     -4.63***   -2.98***

In [51]:
print(results_table.as_latex())

\begin{table}
\caption{Indonesia : Mincer Return - Results from Earnings Regressions}
\label{}
\begin{center}
\begin{tabular}{lllll}
\hline
                          & 1976(unadj.) & 1976(adj.) & 1995(unadj.) & 1995(adj.)  \\
\hline
const                     & 15.08***     & 15.14***   & 15.41***     & 15.66***    \\
                          & (0.01)       & (0.02)     & (0.01)       & (0.01)      \\
sex\_female               & 0.25***      & 0.23***    & 0.28***      & 0.23***     \\
                          & (0.02)       & (0.02)     & (0.01)       & (0.01)      \\
yrschool                  &              & -0.02***   &              & -0.05***    \\
                          &              & (0.00)     &              & (0.00)      \\
indgen\_Industry          & -2.95***     & -2.93***   & -2.10***     & -1.97***    \\
                          & (0.03)       & (0.03)     & (0.01)       & (0.01)      \\
indgen\_Unskilled service & -1.27***     & -1.24***   & -0.37***     & -0.24***